In [3]:
from dotenv import load_dotenv
import os
import json

load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")

In [5]:
def load_latest_current_issues():
    """최신 현재이슈 JSON 파일을 자동으로 로드"""
    import glob
    import os
    from langchain.schema import Document
    
    json_files = glob.glob("data2/*_BigKinds_current_issues.json")
    if not json_files:
        raise FileNotFoundError("현재이슈 JSON 파일을 찾을 수 없습니다.")
    
    latest_file = max(json_files, key=os.path.getctime)
    print(f"📂 로드하는 파일: {latest_file}")
    
    with open(latest_file, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    issues = data["issues"]
    docs = [
        Document(
            page_content=f"{item['제목']}\n{item['내용']}",
            metadata={"이슈번호": item["이슈번호"], "제목": item["제목"]}
        )
        for item in issues
    ]
    
    return docs

# 사용법
docs = load_latest_current_issues()

📂 로드하는 파일: data2\2025.07.23_12.07.17_BigKinds_current_issues.json


In [ ]:

len(docs)   # csv 행이 1205개 있다는 것

10

In [ ]:
# embedding 모델 객체 생성
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


In [ ]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 클라이언트를 초기화(객체생성)
pc = Pinecone(api_key=PINECONE_API_KEY)

In [ ]:
# split하기
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정 (예: 1000자씩 분할)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450, 
    chunk_overlap=45,
    # length_function=tiktoken_len,  # 토큰 기반 길이 측정    
    length_function=len,  # 문자수   
    separators=["\n\n", "\n", " ", ""]
    )

# 문서를 분할
chunks = text_splitter.split_documents(docs)

In [ ]:
len(chunks)

10